In [11]:
# CONFIG
import os
from dotenv import load_dotenv

load_dotenv()

# Airtable configuration
AIRTABLE_API_KEY = os.getenv('AIRTABLE_API_KEY')
AIRTABLE_BASE_ID = os.getenv('AIRTABLE_BASE_ID')
AIRTABLE_CD_TABLE = os.getenv('AIRTABLE_CD_TABLE')

# Data Retrieval

In [13]:
import pandas as pd
from pyairtable import Api

# data processing
def remove_fields_prefix(df):
    df.columns = df.columns.str.replace('fields.', '')
    return df

def process_list_like_values(df):
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    return df

# data fetching
def fetch_data(table_id):
    api = Api(AIRTABLE_API_KEY)
    table = api.table(AIRTABLE_BASE_ID, table_id)
    records = table.all()

    if records:
        df = pd.json_normalize([record['fields'] for record in records])  # Directly normalize 'fields'
        df = remove_fields_prefix(df)
        df = process_list_like_values(df)
        return df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no records

# implementation
df = fetch_data(AIRTABLE_CD_TABLE)

columns_to_check = [
    "Relevant facts / Summary of the case", 
    "Relevant rules of law", 
    "Choice of law issue", 
    "Court's position"
]

df = df.dropna(subset=columns_to_check)

cases = df['Case'].to_list()
abstracts = df['Abstract'].to_list()
texts = df['Content'].to_list()
facts = df['Relevant facts / Summary of the case'].to_list()
rules = df['Relevant rules of law'].to_list()
issues = df['Choice of law issue'].to_list()
positions = df["Court's position"].to_list()

# Case Analysis

In [17]:
# select one single court decision

cd = texts[0]

In [14]:
#utils

import subprocess

def prompt_llama(prompt_text):
    # Pass the prompt text directly as input without flags
    process = subprocess.run(
        ["ollama", "run", "llama3.1"],
        input=prompt_text,  # Provide the prompt directly
        capture_output=True,
        text=True
    )

    # Handle errors if they occur
    if process.returncode != 0:
        print("Error:", process.stderr)
        return None
    
    # Return the model's output
    return process.stdout.strip()

## 0. Abstract

In [18]:
# extract abstracts from texts
prompt_abstract = f"""Here is the text of a Court Decision:
                {cd}
                Find the official abstract stated in the case, usually right at the beginning. This task is successfully accomplished if it is a precise "copy-paste" of the Regeste paragraph of the file. This term varies according to country to country (e.g., Brazil: Ementa).  The translation of certain legal concepts can be challenging. It is a good abstract if it conveys the same meaning (not necessarily obtained via a literal translation).

                Otherwise, this has to return a general description of the information in the file. It's successfully accomplished if (i) it is concise; (ii) it manages to condense all the key details (topic, provisions, information about the legal dispute) in a single paragraph or less. 
                """
abstract_l3 = prompt_llama(prompt_abstract)
print(abstract_l3)

Here is a possible abstract based on the provided text:

"Dispute over notice of termination: The Swiss Federal Supreme Court upholds a cantonal court's decision that an employee was validly dismissed verbally during an interview with two representatives, despite her claims to the contrary and allegations of arbitrariness. The court concludes that the employee should have inferred a representative relationship from the circumstances and that the dismissal was clearly communicated."


## 1. Relevant Facts

In [ ]:
# relevant facts

## 2. Relevant Rules of Law

In [ ]:
# relevant rules of law

## 3. Choice of Law Issue

In [ ]:
# choice of law issue

## 4. Court's Position

In [ ]:
# court's position